In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

See the discussion here:

https://github.com/run-llama/llama_index/discussions/13431

You should use ReactAgentWorker - the function calling agent worker is only for LLMs with a specific API built-in for tool calling (this includes openai, anthropic, and mistral)



In [24]:
from llama_index.core.tools import FunctionTool
from llama_index.llms.ollama import Ollama
from llama_index.core.agent import ReActAgent
from tavily import TavilyClient 
import json
import requests
from collections import Counter


# web_search

def web_search(query: str) -> str:
    """
    Performs a web search using the Tavily API and returns the context string.

    Parameters:
    - query (str): The search query.

    Returns:
    - str: The context string from the Tavily API or an error message.
    """
    try:
        # Step 1: Instantiate the TavilyClient
        tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

        # Step 2: Execute the search query
        context = tavily_client.get_search_context(query=query)

        # Step 3: Return the context
        return f"**Web Search Context:**\n{context}"
    except Exception as e:
        return f"Error performing web search: {str(e)}"

# define sample Tool
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


# weather functions

def update_weather(location: str) -> str:
    api_key = os.environ["OPENWEATHERMAP_API_KEY"]
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {"q": location, "appid": api_key, "units": "imperial"}
    response = requests.get(base_url, params=params)
    weather_data = response.json()

    if response.status_code != 200:
        return f"Error fetching weather data: {weather_data.get('message', 'Unknown error')}"

    lon = weather_data["coord"]["lon"]
    lat = weather_data["coord"]["lat"]
    main = weather_data["weather"][0]["main"]
    feels_like = weather_data["main"]["feels_like"]
    temp_min = weather_data["main"]["temp_min"]
    temp_max = weather_data["main"]["temp_max"]
    pressure = weather_data["main"]["pressure"]
    visibility = weather_data["visibility"]
    wind_speed = weather_data["wind"]["speed"]
    wind_deg = weather_data["wind"]["deg"]
    sunrise = datetime.fromtimestamp(weather_data["sys"]["sunrise"]).strftime(
        "%H:%M:%S"
    )
    sunset = datetime.fromtimestamp(weather_data["sys"]["sunset"]).strftime("%H:%M:%S")
    temp = weather_data["main"]["temp"]
    humidity = weather_data["main"]["humidity"]
    condition = weather_data["weather"][0]["description"]

    return f"""**Weather in {location}:**
- **Coordinates:** (lon: {lon}, lat: {lat})
- **Temperature:** {temp:.2f}°F (Feels like: {feels_like:.2f}°F)
- **Min Temperature:** {temp_min:.2f}°F, **Max Temperature:** {temp_max:.2f}°F
- **Humidity:** {humidity}%
- **Condition:** {condition.capitalize()}
- **Pressure:** {pressure} hPa
- **Visibility:** {visibility} meters
- **Wind Speed:** {wind_speed} m/s, **Wind Direction:** {wind_deg}°
- **Sunrise:** {sunrise}, **Sunset:** {sunset}"""


def update_weather_forecast(location: str) -> str:
    """Fetches the weather forecast for a given location and returns a formatted string
    Parameters:
    - location: the search term to find weather information
    Returns:
    A formatted string containing the weather forecast data
    """

    api_key = os.environ["OPENWEATHERMAP_API_KEY"]
    base_url = "http://api.openweathermap.org/data/2.5/forecast"
    params = {
        "q": location,
        "appid": api_key,
        "units": "imperial",
        "cnt": 40,  # Request 40 data points (5 days * 8 three-hour periods)
    }
    response = requests.get(base_url, params=params)
    weather_data = response.json()
    if response.status_code != 200:
        return f"Error fetching weather data: {weather_data.get('message', 'Unknown error')}"

    # Organize forecast data per date
    forecast_data = {}
    for item in weather_data["list"]:
        dt_txt = item["dt_txt"]  # 'YYYY-MM-DD HH:MM:SS'
        date_str = dt_txt.split(" ")[0]  # 'YYYY-MM-DD'
        time_str = dt_txt.split(" ")[1]  # 'HH:MM:SS'
        forecast_data.setdefault(date_str, [])
        forecast_data[date_str].append(
            {
                "time": time_str,
                "temp": item["main"]["temp"],
                "feels_like": item["main"]["feels_like"],
                "humidity": item["main"]["humidity"],
                "pressure": item["main"]["pressure"],
                "wind_speed": item["wind"]["speed"],
                "wind_deg": item["wind"]["deg"],
                "condition": item["weather"][0]["description"],
                "visibility": item.get(
                    "visibility", "N/A"
                ),  # sometimes visibility may be missing
            }
        )

    # Process data to create daily summaries
    daily_summaries = {}
    for date_str, forecasts in forecast_data.items():
        temps = [f["temp"] for f in forecasts]
        feels_likes = [f["feels_like"] for f in forecasts]
        humidities = [f["humidity"] for f in forecasts]
        pressures = [f["pressure"] for f in forecasts]
        wind_speeds = [f["wind_speed"] for f in forecasts]
        conditions = [f["condition"] for f in forecasts]

        min_temp = min(temps)
        max_temp = max(temps)
        avg_temp = sum(temps) / len(temps)
        avg_feels_like = sum(feels_likes) / len(feels_likes)
        avg_humidity = sum(humidities) / len(humidities)
        avg_pressure = sum(pressures) / len(pressures)
        avg_wind_speed = sum(wind_speeds) / len(wind_speeds)

        # Find the most common weather condition
        condition_counts = Counter(conditions)
        most_common_condition = condition_counts.most_common(1)[0][0]

        daily_summaries[date_str] = {
            "min_temp": min_temp,
            "max_temp": max_temp,
            "avg_temp": avg_temp,
            "avg_feels_like": avg_feels_like,
            "avg_humidity": avg_humidity,
            "avg_pressure": avg_pressure,
            "avg_wind_speed": avg_wind_speed,
            "condition": most_common_condition,
        }

    # Build the formatted string
    city_name = weather_data["city"]["name"]
    ret_str = f"**5-Day Weather Forecast for {city_name}:**\n"

    for date_str in sorted(daily_summaries.keys()):
        summary = daily_summaries[date_str]
        ret_str += f"\n**{date_str}:**\n"
        ret_str += f"- **Condition:** {summary['condition'].capitalize()}\n"
        ret_str += f"- **Min Temperature:** {summary['min_temp']:.2f}°F\n"
        ret_str += f"- **Max Temperature:** {summary['max_temp']:.2f}°F\n"
        ret_str += f"- **Average Temperature:** {summary['avg_temp']:.2f}°F (Feels like {summary['avg_feels_like']:.2f}°F)\n"
        ret_str += f"- **Humidity:** {summary['avg_humidity']:.0f}%\n"
        ret_str += f"- **Pressure:** {summary['avg_pressure']:.0f} hPa\n"
        ret_str += f"- **Wind Speed:** {summary['avg_wind_speed']:.2f} m/s\n"

    return ret_str

web_search_tool = FunctionTool.from_defaults(fn=web_search)
multiply_tool = FunctionTool.from_defaults(fn=multiply)
update_weather_tool = FunctionTool.from_defaults(fn=update_weather)
update_weather_forecast_tool = FunctionTool.from_defaults(fn=update_weather_forecast)


# initialize llm
llm = Ollama(model="llama3", temperature=0, format="json")

# initialize ReAct agent
agent = ReActAgent.from_tools([multiply_tool, web_search_tool,
                    update_weather_tool, update_weather_forecast_tool], llm=llm, verbose=True,
                             max_iterations=5)

In [25]:
response = agent.chat("What is the current weather in Newton, Massachusetts?")

> Running step d033e597-16d2-45cc-95dd-d6c13ef83dc2. Step input: What is the current weather in Newton, Massachusetts?
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: update_weather_forecast
Action Input: {'location': 'Newton, Massachusetts'}
Observation: **5-Day Weather Forecast for Newton:**

**2024-10-19:**
- **Condition:** Clear sky
- **Min Temperature:** 44.87°F
- **Max Temperature:** 69.89°F
- **Average Temperature:** 54.93°F (Feels like 54.00°F)
- **Humidity:** 60%
- **Pressure:** 1032 hPa
- **Wind Speed:** 3.00 m/s

**2024-10-20:**
- **Condition:** Clear sky
- **Min Temperature:** 48.02°F
- **Max Temperature:** 73.58°F
- **Average Temperature:** 57.42°F (Feels like 55.45°F)
- **Humidity:** 51%
- **Pressure:** 1026 hPa
- **Wind Speed:** 5.81 m/s

**2024-10-21:**
- **Condition:** Clear sky
- **Min Temperature:** 52.48°F
- **Max Temperature:** 77.16°F
- **Average Temperature:** 61.77°F (Feels like 60.05°F)
- **Humi

In [26]:
print(response)

The current weather in Newton, Massachusetts is clear sky with a temperature of around 54°F (Feels like 54°F).
